# Second Model
Prepare the Data

In [ ]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob



In [ ]:
# setup data 
image_list = []
    # -3 means back one directory
# filename/shape/type/label
for filepath in glob.glob('images/*/*/*.png', recursive=True):
     filename = filepath.split("\\")[-1]
     sign_shape = filepath.split("\\")[-2]
     sign_type = filepath.split("\\")[-3]
     
     image_list.append((filepath,sign_shape,sign_type))
     
data = pd.DataFrame(data=image_list, columns=['image_path','sign_type', 'sign_shape'])


Dataframes for training and validation

In [ ]:
r_inx = np.random.choice(100, 4)
rand_data = data.loc[r_inx,'image_path']


d = {'rightofway':0, 'stop':1, 'bicycle':2,
     'limitedtraffic':3, 'noentry':4, 'noparking':5,
     'roundabout':6, 'speed':7,'trafficdirective':8,
     'traveldirection':9,'continue':10,'crossing':11,
     'laneend':12, 'parking':13, 'giveway':14, 'warning':15}
data['type_num'] = data['sign_type'].map(d, na_action='ignore')

train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])


N_train_images = train.shape[0]
N_val_images = validate.shape[0]
N_test_images = test.shape[0]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last',
                                   rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,fill_mode='nearest')
                                
                                   
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last',
                                   rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')


batch_size = 300

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        directory='./',
        x_col="image_path",
        y_col="sign_type",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=validate,
        directory='./',
        x_col="image_path",
        y_col="sign_type",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test,
        directory='./',
        x_col="image_path",
        y_col="sign_type",
        target_size=(28, 28),
        batch_size=batch_size,
        class_mode=None
)


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Lambda
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.metrics import categorical_accuracy, sparse_categorical_crossentropy, sparse_categorical_accuracy


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import regularizers, optimizers
from keras.layers import GaussianNoise

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = Sequential()
chanDim = -1

# input
model.add(Input(shape=(28, 28, 3)))
model.add(Lambda(lambda x: tf.expand_dims(x[:,:,:,0], -1, name=None))) 
# this is a workaround. Dataloader automatically read one channel image as 3 channel 
#and we use Lambda layer to revert this back. Lambda layer can be used for operation 
#that does not involve trainianble weights

model.add(Conv2D(8, (5, 5), padding="same",kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# first set of (CONV => RELU => CONV => RELU) * 2 => POOL
model.add(Conv2D(16, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(16, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
# second set of (CONV => RELU => CONV => RELU) * 2 => POOL
model.add(Conv2D(32, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))


# first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# second set of FC => RELU layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(16))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=[categorical_accuracy])

model.summary()

In [ ]:
def calculate_losses(model_in, data_generator_in, N_images, batch_size_):
    loss_hold = []
    acc_hold = []
    batches = 0

    # iterate over each batch
    for x,y in data_generator_in:
        loss, acc = model_in.evaluate(x, y, verbose=0)
        loss_hold.append(loss)
        acc_hold.append(acc)
        batches += 1
        if batches >= N_images / batch_size_:
            # we need to break the loop by hand because the generator loops indefinitely
            break

    return np.mean(loss_hold), np.mean(acc_hold)


def train_model_two(model_, num_epoch=30, verbose=False):
    res = []
    for e in range(num_epoch):
        # print('Epoch', e)
        batches = 0

        loss_ = []
        acc_ = []

         # iterate over each batch
        for x,y in train_generator:
            loss, acc = model_.train_on_batch(x, y) # Update weights and return train loss, acc per batch
            loss_.append(loss)
            acc_.append(acc)
            batches += 1
            if batches >= N_train_images / batch_size:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break
        loss_ = np.mean(loss_)
        acc_ = np.mean(acc_)

        loss, acc = calculate_losses(model_, validation_generator, N_val_images, batch_size)
        if verbose:
            print("Training epoch {}: Loss = {}, Accuracy = {}".format(e, loss_, acc_))
            print("Validation epoch {}: Loss = {}, Accuracy = {}".format(e, loss, acc))

        res.append((e, loss_, acc_, loss, acc))

    model_.save('model.h5')
    print('Model saved')

    return np.asarray(res)



res_two = train_model_two(model, num_epoch=30, verbose=True)



def plot_results(res):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(res[:,0], res[:,1], 'r-')
    plt.plot(res[:,0], res[:,3], 'b-')
    plt.legend(['Train', 'Validation'])
    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.ylim([0, np.max([5., np.max(res[:,1]), np.max(res[:,3])])])

    plt.subplot(1,2,2)
    plt.plot(res[:,0], res[:,2], 'r-')
    plt.plot(res[:,0], res[:,4], 'b-')
    plt.legend(['Train', 'Validation'])
    plt.xlabel('epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, np.max([1., np.max(res[:,2]), np.max(res[:,4])])])


Predict Model 2

In [ ]:
plot_results(res_two)



pred = model.predict(test_generator)
print(np.argmax(pred[0]))

predicted_class_indices=np.argmax(pred,axis=1)


labels = train_generator.filenames
labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]


filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)


